# Simple Moving Average Plot

Little script to (1) request realtime and historical global equity data with Alpha Vantage API, (2) evaluete 2 different time periods moving average and (3) plot it for comparison.
Feel free to use, improve and change data parameters.

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime


#request function
def request(ticker):
    url = "https://www.alphavantage.co/query"

    api_key = "###########" #insert your api key. Claim your: https://www.alphavantage.co/support/#api-key

    data = { "function": 'TIME_SERIES_DAILY', #change parameters to change temporal resolution. doc:https://www.alphavantage.co/documentation/)%20for%20TIME_SERIES_INTRADAY.'
             "symbol": ticker, 
             "apikey": api_key,
            'outputsize' : 'full'
           }
    page = requests.get(url, params = data)
    
    return pd.DataFrame(page.json()['Time Series (Daily)']).T

#converting to time series function
def todate(timeseries):
    x=[]
    y=[]
    for i in range(len(timeseries)):
        if(np.isnan(float(timeseries[i]))==False):
            x.append(datetime.strptime(timeseries.index[i],'%Y-%m-%d'))
            y.append(float(timeseries[i]))
    timeseries = pd.Series(y,x)
    return (timeseries)

#evaluate moving averages function
def SMA_evaluation(df,KMAS,KMAL):
    MAS = df['4. close'].rolling(window=KMAS).mean()
    MAS = todate(MAS)
    MAL = df['4. close'].rolling(window=KMAL).mean()
    MAL = todate(MAL)
    P = df['4. close']
    P = todate(P)
    return MAS, MAL, P

#plot function
def SMA_plot(MAS,MAL,P,ticker,KMAS,KMAL):
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    plt.plot(P.index, P, label='Price')
    plt.plot(MAS.index, MAS, label='{} time periods Moving Average'.format(KMAS))
    plt.plot(MAL.index, MAL, label='{} time periods Moving Average'.format(KMAL))
    plt.legend()
    plt.axis([P.index[-300], P.index[-1], min(P[-300:])-np.std(P[-300:]), max(P[-300:])+np.std(P[-300:])]) #I'm ploting 300 time periods
    plt.title('{}'.format(ticker))
    plt.ylabel('Price ($)')

#main
def SMA_Comparison(ticker,KMAS=28,KMAL=72):
    df = request(ticker)
    MAS,MAL,P = SMA_evaluation(df,KMAS,KMAL)
    SMA_plot(MAS,MAL,P,ticker,KMAS,KMAL)

    
#parameters:
ticker = '####' #stock ticker
KMAS = 28 #short-term moving average number of time periods
KMML = 72 #long-term  moving average number of time periods

SMA_Comparison(ticker,KMAS,KMAL)